### Appendix D: UVM Quick Reference

#### UVM Base Classes Hierarchy

##### Core Base Classes
```systemverilog
uvm_object
├── uvm_transaction
├── uvm_component
│   ├── uvm_test
│   ├── uvm_env
│   ├── uvm_agent
│   ├── uvm_monitor
│   ├── uvm_driver
│   ├── uvm_sequencer
│   └── uvm_scoreboard
├── uvm_sequence_item
└── uvm_sequence
```

#### UVM Macros Quick Reference

##### Registration Macros
```systemverilog
// Component registration
`uvm_component_utils(class_name)
`uvm_component_utils_begin(class_name)
`uvm_component_utils_end

// Object registration
`uvm_object_utils(class_name)
`uvm_object_utils_begin(class_name)
`uvm_object_utils_end

// Field automation
`uvm_field_int(field_name, flag)
`uvm_field_string(field_name, flag)
`uvm_field_object(field_name, flag)
`uvm_field_array_int(field_name, flag)
```

##### Messaging Macros
```systemverilog
`uvm_info(ID, MSG, VERBOSITY)
`uvm_warning(ID, MSG)
`uvm_error(ID, MSG)
`uvm_fatal(ID, MSG)

// Verbosity levels
UVM_NONE, UVM_LOW, UVM_MEDIUM, UVM_HIGH, UVM_FULL, UVM_DEBUG
```

##### Factory Macros
```systemverilog
// Type override
`uvm_component_registry(class_name, "string_name")
`uvm_object_registry(class_name, "string_name")

// Creation
`uvm_create(item)
`uvm_create_on(item, seqr)
`uvm_send(item)
`uvm_send_pri(item, priority)
```

#### Common UVM Patterns

##### Component Constructor Pattern
```systemverilog
class my_component extends uvm_component;
  `uvm_component_utils(my_component)
  
  function new(string name, uvm_component parent);
    super.new(name, parent);
  endfunction
endclass
```

##### Object Constructor Pattern
```systemverilog
class my_transaction extends uvm_sequence_item;
  `uvm_object_utils(my_transaction)
  
  function new(string name = "my_transaction");
    super.new(name);
  endfunction
endclass
```

##### Build Phase Pattern
```systemverilog
function void build_phase(uvm_phase phase);
  super.build_phase(phase);
  
  // Configuration
  if (!uvm_config_db#(my_config)::get(this, "", "config", m_config))
    `uvm_fatal("CONFIG", "Cannot get config")
    
  // Component creation
  m_agent = my_agent::type_id::create("m_agent", this);
endfunction
```

##### Connect Phase Pattern
```systemverilog
function void connect_phase(uvm_phase phase);
  super.connect_phase(phase);
  
  // Port connections
  driver.seq_item_port.connect(sequencer.seq_item_export);
  monitor.analysis_port.connect(scoreboard.analysis_export);
endfunction
```

#### UVM Phases

##### Phase Execution Order
1. **build_phase** - Component construction and configuration
2. **connect_phase** - Port and export connections
3. **end_of_elaboration_phase** - Final setup before simulation
4. **start_of_simulation_phase** - Simulation start notifications
5. **run_phase** - Main simulation execution (time-consuming)
6. **extract_phase** - Data extraction from components
7. **check_phase** - Checking and validation
8. **report_phase** - Final reporting
9. **final_phase** - Cleanup activities

##### Phase Methods
```systemverilog
// Function phases (zero time)
virtual function void build_phase(uvm_phase phase);
virtual function void connect_phase(uvm_phase phase);
virtual function void end_of_elaboration_phase(uvm_phase phase);
virtual function void start_of_simulation_phase(uvm_phase phase);
virtual function void extract_phase(uvm_phase phase);
virtual function void check_phase(uvm_phase phase);
virtual function void report_phase(uvm_phase phase);
virtual function void final_phase(uvm_phase phase);

// Task phase (time-consuming)
virtual task run_phase(uvm_phase phase);
```

#### Configuration Database

##### Setting Configuration
```systemverilog
// In test or higher level component
uvm_config_db#(my_config)::set(this, "env.agent*", "config", cfg);
uvm_config_db#(int)::set(this, "*", "num_transactions", 100);
uvm_config_db#(virtual my_if)::set(null, "*", "vif", vif);
```

##### Getting Configuration
```systemverilog
// In component
my_config cfg;
if (!uvm_config_db#(my_config)::get(this, "", "config", cfg))
  `uvm_fatal("CONFIG", "Cannot get configuration")

virtual my_if vif;
if (!uvm_config_db#(virtual my_if)::get(this, "", "vif", vif))
  `uvm_fatal("VIF", "Cannot get virtual interface")
```

#### Factory Operations

##### Type Overrides
```systemverilog
// In test
function void build_phase(uvm_phase phase);
  // Override specific instance
  factory.set_inst_override_by_type(base_test::get_type(),
                                   extended_test::get_type(),
                                   "uvm_test_top");
  
  // Override all instances of a type
  factory.set_type_override_by_type(base_driver::get_type(),
                                   extended_driver::get_type());
endfunction
```

##### Object Creation
```systemverilog
// Using factory
my_transaction tr = my_transaction::type_id::create("tr");

// Direct creation (bypasses factory)
my_transaction tr = new("tr");
```

#### Sequence Patterns

##### Basic Sequence
```systemverilog
class my_sequence extends uvm_sequence#(my_transaction);
  `uvm_object_utils(my_sequence)
  
  function new(string name = "my_sequence");
    super.new(name);
  endfunction
  
  virtual task body();
    my_transaction req;
    
    repeat(10) begin
      `uvm_create(req)
      req.randomize();
      `uvm_send(req)
    end
  endtask
endclass
```

##### Virtual Sequence
```systemverilog
class my_virtual_sequence extends uvm_sequence;
  `uvm_object_utils(my_virtual_sequence)
  
  my_sequencer seqr1, seqr2;
  
  virtual task body();
    fork
      begin
        my_seq1 seq = my_seq1::type_id::create("seq");
        seq.start(seqr1);
      end
      begin
        my_seq2 seq = my_seq2::type_id::create("seq");
        seq.start(seqr2);
      end
    join
  endtask
endclass
```

#### Analysis Components

##### Analysis Port Declaration and Connection
```systemverilog
// In monitor
uvm_analysis_port#(my_transaction) analysis_port;

// In constructor
analysis_port = new("analysis_port", this);

// In monitor's main task
analysis_port.write(tr);

// In connect phase of parent
monitor.analysis_port.connect(scoreboard.analysis_export);
```

##### Scoreboard with Analysis Export
```systemverilog
class my_scoreboard extends uvm_scoreboard;
  `uvm_component_utils(my_scoreboard)
  
  uvm_analysis_export#(my_transaction) analysis_export;
  uvm_tlm_analysis_fifo#(my_transaction) analysis_fifo;
  
  function new(string name, uvm_component parent);
    super.new(name, parent);
  endfunction
  
  function void build_phase(uvm_phase phase);
    analysis_export = new("analysis_export", this);
    analysis_fifo = new("analysis_fifo", this);
  endfunction
  
  function void connect_phase(uvm_phase phase);
    analysis_export.connect(analysis_fifo.analysis_export);
  endfunction
  
  task run_phase(uvm_phase phase);
    my_transaction tr;
    forever begin
      analysis_fifo.get(tr);
      // Process transaction
    end
  endtask
endclass
```

#### Common UVM TLM Interfaces

##### Blocking Interfaces
```systemverilog
// put/get interfaces
uvm_blocking_put_port#(T)
uvm_blocking_get_port#(T)
uvm_blocking_peek_port#(T)

// Transport interfaces  
uvm_blocking_transport_port#(REQ, RSP)
uvm_blocking_master_port#(REQ, RSP)
uvm_blocking_slave_port#(REQ, RSP)
```

##### Non-blocking Interfaces
```systemverilog
// try_put/try_get interfaces
uvm_nonblocking_put_port#(T)
uvm_nonblocking_get_port#(T)
uvm_nonblocking_peek_port#(T)

// Non-blocking transport
uvm_nonblocking_transport_port#(REQ, RSP)
```

##### Combined Interfaces
```systemverilog
// Blocking + Non-blocking
uvm_put_port#(T)
uvm_get_port#(T)
uvm_peek_port#(T)
uvm_transport_port#(REQ, RSP)
```

#### Synchronization and Communication

##### Events
```systemverilog
// Declaring events
uvm_event start_event;
uvm_event_pool event_pool;

// Using events
start_event = event_pool.get("start_event");
start_event.wait_trigger();
start_event.trigger();
```

##### Barriers
```systemverilog
// Creating barrier
uvm_barrier barrier = new("barrier", 3); // 3 participants

// Waiting for barrier
barrier.wait_for();
```

##### Callbacks
```systemverilog
// Define callback class
class my_driver_cb extends uvm_callback;
  virtual task pre_send(my_driver driver, my_transaction tr);
  endtask
endclass

// Register callback in driver
`uvm_register_cb(my_driver, my_driver_cb)

// Use callback in driver
`uvm_do_callbacks(my_driver, my_driver_cb, pre_send(this, tr))
```

#### Field Automation Flags

##### Common Flags
```systemverilog
UVM_DEFAULT     // Default behavior (compare, copy, print, pack)
UVM_ALL_ON      // All operations enabled
UVM_NOPACK      // Exclude from pack/unpack
UVM_NOCOMPARE   // Exclude from compare
UVM_NOCOPY      // Exclude from copy
UVM_NOPRINT     // Exclude from print
UVM_DEC         // Print in decimal
UVM_HEX         // Print in hexadecimal
UVM_BIN         // Print in binary
UVM_OCT         // Print in octal
```

#### Resource Database

##### Setting Resources
```systemverilog
uvm_resource_db#(int)::set("*", "timeout", 1000);
uvm_resource_db#(string)::set("test", "mode", "regression");
```

##### Getting Resources
```systemverilog
int timeout;
if (uvm_resource_db#(int)::read_by_name("*", "timeout", timeout))
  `uvm_info("TIMEOUT", $sformatf("Timeout set to %0d", timeout), UVM_LOW)
```

#### Command Line Options

##### UVM Command Line Arguments
```bash
# Verbosity control
+UVM_VERBOSITY=UVM_HIGH
+UVM_VERBOSITY=UVM_MEDIUM

# Timeout
+UVM_TIMEOUT=10000

# Maximum errors
+UVM_MAX_QUIT_COUNT=5

# Configuration
+uvm_set_config_int=*,timeout,5000
+uvm_set_config_string=*,mode,debug

# Factory overrides
+uvm_set_type_override=base_test,extended_test
+uvm_set_inst_override=uvm_test_top,base_test,extended_test
```

#### Debug and Reporting

##### Factory Debug
```systemverilog
// Print factory configuration
factory.print();

// Print all registered types
uvm_factory::get().print();
```

##### Topology Debug
```systemverilog
// Print testbench hierarchy
uvm_top.print_topology();

// Print configuration database
uvm_config_db#(int)::dump();
```

##### Phase Debug
```systemverilog
// Set phase timeout
uvm_top.set_timeout(10ms);

// Print phase information
uvm_phase::get_current_phase().print();
```

#### Memory and Register Model Integration

##### Register Model Basic Usage
```systemverilog
// Declare register model
my_reg_model reg_model;

// Build phase
reg_model = my_reg_model::type_id::create("reg_model");
reg_model.build();
reg_model.configure(null, "");

// Using register model
uvm_status_e status;
uvm_reg_data_t data;

// Write register
reg_model.my_reg.write(status, 32'hDEADBEEF);

// Read register  
reg_model.my_reg.read(status, data);
```

##### Memory Model Usage
```systemverilog
// Write to memory
reg_model.my_mem.write(status, addr, data);

// Read from memory
reg_model.my_mem.read(status, addr, data);

// Burst operations
reg_model.my_mem.burst_write(status, addr, data_array);
reg_model.my_mem.burst_read(status, addr, data_array);
```

---

*This quick reference covers the most commonly used UVM constructs and patterns. For detailed information, refer to the UVM User Guide and IEEE 1800.2 Standard.*